In [4]:
import os
import requests
import gradio as gr
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# Load environment variables
load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
WEATHER_API_KEY = os.getenv("WEATHER_API_KEY")
NEWS_API_KEY = os.getenv("NEWS_API_KEY")

# Setup LLM
llm = ChatGroq(temperature=0.3, model_name="llama3-8b-8192")

# Prompt template for news summarization
news_prompt = PromptTemplate(
    input_variables=["news"],
    template="""
You are an intelligent assistant. Summarize the following news headlines in a concise and engaging manner for a general audience:

{news}
"""
)

# Prompt template for weather beautification
weather_prompt = PromptTemplate(
    input_variables=["city", "weather"],
    template="""
You are a friendly assistant. Based on the weather info provided below, give a natural language response about the current weather in {city}.

Weather data:
{weather}
"""
)

# Chains
weather_chain = LLMChain(llm=llm, prompt=weather_prompt)
news_chain = LLMChain(llm=llm, prompt=news_prompt)

# Function to fetch weather data
def get_weather(city):
    url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={WEATHER_API_KEY}&units=metric"
    res = requests.get(url)
    if res.status_code != 200:
        return None
    data = res.json()
    return f"Temperature: {data['main']['temp']}°C, Condition: {data['weather'][0]['description'].capitalize()}"

# Function to fetch news headlines
def get_top_news():
    url = f"https://newsapi.org/v2/top-headlines?language=en&pageSize=5&apiKey={NEWS_API_KEY}"
    res = requests.get(url)
    if res.status_code != 200:
        return None
    articles = res.json().get("articles", [])
    return "\n".join([f"{i+1}. {a['title']}" for i, a in enumerate(articles)])

# Main function to combine everything
def assistant(city):
    weather_raw = get_weather(city)
    if not weather_raw:
        weather_result = "❌ Could not fetch weather. Check city name or API key."
    else:
        weather_result = weather_chain.run(city=city, weather=weather_raw)

    news_raw = get_top_news()
    if not news_raw:
        news_result = "❌ Could not fetch news. Check your NewsAPI key."
    else:
        news_result = news_chain.run(news=news_raw)

    return weather_result, news_result

# Gradio UI
iface = gr.Interface(
    fn=assistant,
    inputs=gr.Textbox(placeholder="Enter city name", label="City"),
    outputs=[
        gr.Textbox(label="🌤 Live Weather Update"),
        gr.Textbox(label="📰 Global Trending News Summary")
    ],
    title="🌍 Live Weather + Global News Assistant",
    description="Enter a city name to get live weather and summarized top global news."
)

if __name__ == "__main__":
    iface.launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.
